In [9]:
import pandas as pd
import warnings
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import psycopg2
import time

In [2]:
warnings.filterwarnings("ignore")
conn = psycopg2.connect(database="postgres", user="postgres",
                        password="password", host="127.0.0.1", port="5432")

In [36]:
cur = conn.cursor()
cur.execute('''SELECT to_timestamp(floor((extract('epoch' from Created_at) / 30 )) * 30) AT TIME ZONE 'UTC' time_ , avg(price) price_
                            FROM BT_Price
                            WHERE created_at <= date_trunc('hour',  now() + interval '1 hour') and created_at >= (now() - interval '1 hour')
                            GROUP BY time_
                            ORDER BY time_''')
rows = cur.fetchall()
conn.commit()

df = pd.DataFrame(rows, columns=['time', 'price'])
results = []
history = [x for x in df["price"].values]

for i in range(0, 5):
    current_res = {}
    current_time = datetime.now() + timedelta(seconds=(i * 30))
    model = ARIMA(history, order=(0, 3, 2))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    current_res["price"] = yhat
    if current_time.second < 30:
        newdatetime = current_time.replace(second=30, microsecond=0)
    elif newdatetime.minute + 1 <= 59:
        newdatetime = current_time.replace(minute=(current_time.minute + (1)), second=0, microsecond=0)
    elif(newdatetime.hour != 23):
        newdatetime = current_time.replace(hour=(current_time.hour + 1), minute=1, second=0, microsecond=0)
    else:
        newdatetime = newdatetime.replace(hour=(current_time.hour + 1), minute=1, second=0, microsecond=0)
    current_res["datetime"] = newdatetime
    results.append(current_res) # SQL
    history.append(yhat)

In [37]:
for res in results:
    print(res)

{'price': 34458.72472059648, 'datetime': datetime.datetime(2021, 6, 28, 22, 43)}
{'price': 34461.175252766174, 'datetime': datetime.datetime(2021, 6, 28, 22, 43, 30)}
{'price': 34463.73636938098, 'datetime': datetime.datetime(2021, 6, 28, 22, 44)}
{'price': 34466.40876774974, 'datetime': datetime.datetime(2021, 6, 28, 22, 44, 30)}
{'price': 34469.1931112571, 'datetime': datetime.datetime(2021, 6, 28, 22, 45)}
